In [2]:
##All General Import Statements
import pandas as pd
import numpy as np
import math
import random
%matplotlib inline
import random
from matplotlib import pyplot
import os



In [3]:
data = pd.read_csv('/creditcard.csv')

isolation forest

In [6]:
class ExNode:
  def __init__(self, size):
    self.size = size
class InNode:
  def __init__(self, left, right, splitat,splitval):
    self.left = left
    self.right = right
    self.splitat = splitat
    self.splitval = splitval

In [7]:
#X: Input dataset (typically a Pandas DataFrame).
#noOfTrees: The number of isolation trees to construct.
#sampleSize: The number of samples to be drawn randomly for each tree.


def iForest(X, nooftrees, samplesize):
  forest=[]
  h = math.ceil(math.log(samplesize,2))
  for i in range(nooftrees):
    Xtrain = X.sample(samplesize)
    forest.append(buildtree(Xtrain,0,h))
  return forest


In [18]:

def buildtree(Xt, currheight, h):
  if currheight>h:
    return ExNode(len(Xt))
  else:
    # Xt= Xt.reset_index(drop=True)
    Q = Xt.columns
    q = random.choice(Q)
    p = random.choice(Xt[q])
    Xl = Xt[Xt[q]<p]
    Xr = Xt[Xt[q]>=p]

    return InNode(buildtree(Xl,currheight+1,h),buildtree(Xr,currheight+1,h),q,p)


In [9]:
def path_len(X, tree, currheight):
  if isinstance(tree,ExNode):
    return currheight
  a = tree.splitat
  if X[a]<tree.splitval:
    return path_len(X,tree.left,currheight+1)
  else:
    return path_len(X,tree.right,currheight+1)

In [4]:
#run on dataset

df = pd.read_csv('/content/creditcard.csv')
y=df['Class']
df_data=df.drop('Class',axis=1)

In [19]:
sampleSize=100
ifor=iForest(df_data.sample(1000),10,sampleSize)

IndexError: Cannot choose from an empty sequence

In [ ]:
posLenLst=[]
negLenLst=[]

for sim in range(1000):
    ind=random.choice(df_data[y_true==1].index)
    for tree in ifor:
        posLenLst.append(pathLength(df_data.iloc[ind],tree,0))

    ind=random.choice(df_data[y_true==0].index)
    for tree in ifor:
        negLenLst.append(pathLength(df_data.iloc[ind],tree,0))

In [ ]:
bins = np.linspace(0,math.ceil(math.log(sampleSize,2)), math.ceil(math.log(sampleSize,2)))

pyplot.figure(figsize=(12,8))
pyplot.hist(posLenLst, bins, alpha=0.5, label='Anomaly')
pyplot.hist(negLenLst, bins, alpha=0.5, label='Normal')
pyplot.xlabel('Path Length')
pyplot.ylabel('Frequency')
pyplot.legend(loc='upper left')

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.cluster import KMeans

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_data, y_true, test_size=0.3, random_state=42)

In [ ]:
def preprocess(df_data):
    for col in df_data:
        df_data[col]=(df_data[col]-np.min(df_data[col]))/(np.max(df_data[col])-np.min(df_data[col]))
    return

In [ ]:
def train(X,clf,ensembleSize=5,sampleSize=10000):
    mdlLst=[]
    for n in range(ensembleSize):
        X=df_data.sample(sampleSize)
        clf.fit(X)
        mdlLst.append(clf)
    return mdlLst

In [ ]:
def predict(X,mdlLst):
    y_pred=np.zeros(X.shape[0])
    for clf in mdlLst:
        y_pred=np.add(y_pred,clf.decision_function(X).reshape(X.shape[0],))
    y_pred=(y_pred*1.0)/len(mdlLst)
    return y_pred

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix,f1_score

In [ ]:
alg=IsolationForest(n_estimators=100, max_samples='auto', contamination=0.01, \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0,behaviour="new")

In [ ]:
if_mdlLst=train(X_train,alg)

In [ ]:
if_y_pred=predict(X_test,if_mdlLst)
if_y_pred=1-if_y_pred

#Creating class labels based on decision function
if_y_pred_class=if_y_pred.copy()
if_y_pred_class[if_y_pred>=np.percentile(if_y_pred,95)]=1
if_y_pred_class[if_y_pred<np.percentile(if_y_pred,95)]=0

In [ ]:
roc_auc_score(y_test, if_y_pred_class)

In [ ]:
f1_score(y_test, if_y_pred_class)

In [ ]:
if_cm=confusion_matrix(y_test, if_y_pred_class)

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42,n_jobs=-1).fit(X_train)

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=42,n_jobs=-1).fit(X_train)

In [ ]:
X_test_clusters=kmeans.predict(X_test)
X_test_clusters_centers=kmeans.cluster_centers_
dist = [np.linalg.norm(x-y) for x,y in zip(X_test.values,X_test_clusters_centers[X_test_clusters])]

km_y_pred=np.array(dist)
km_y_pred[dist>=np.percentile(dist,95)]=1
km_y_pred[dist<np.percentile(dist,95)]=0

In [ ]:
roc_auc_score(y_test, km_y_pred)

In [ ]:
f1_score(y_test, km_y_pred)

In [ ]:
km_cm=confusion_matrix(y_test, km_y_pred)

In [ ]:
df_cm = pd.DataFrame(km_cm,
                  ['True Normal','True Fraud'],['Pred Normal','Pred Fraud'])
pyplot.figure(figsize = (8,4))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 16},fmt='g')